In [1]:
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 900.9/900.9 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [3]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["UNSLOTH_DISABLE"] = "1"
os.environ["TORCHDYNAMO_DISABLE"] = "1"  
os.environ["TORCH_COMPILE_DISABLE"] = "1"
import unsloth
import torch
import sys
import logging
import evaluate

import pandas as pd
import numpy as np

from unsloth import FastModel, FastLanguageModel
from transformers import AutoTokenizer, AutoModel, AutoConfig, TrainingArguments, Trainer, AutoModelForSequenceClassification, training_args
from datasets import Dataset

from sklearn.model_selection import train_test_split

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-01 13:17:44.724629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762003064.934127      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762003064.995742      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
train = pd.read_csv("/kaggle/input/corpus-imdb/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/corpus-imdb/testData.tsv", header=0, delimiter="\t", quoting=3)

In [5]:
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info(r"running %s" % ''.join(sys.argv))

train, val = train_test_split(train, test_size=.2)

# train = train[0:20]

train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

In [6]:
model_name = 'answerdotai/ModernBERT-large'
# model_name = "microsoft/deberta-v2-xxlarge"
NUM_CLASSES = 2

model, tokenizer = FastModel.from_pretrained(
    model_name=model_name,
    load_in_4bit=False,
    max_seq_length=512,
    dtype=None,
    auto_model=AutoModelForSequenceClassification,
    num_labels=NUM_CLASSES,
    gpu_memory_utilization=0.5  # Reduce if out of memory
)

model = FastModel.get_peft_model(
    model,
    r=16,  # The larger, the higher the accuracy, but might overfit
    lora_alpha=32,  # Recommended alpha == r at least
    lora_dropout=0,
    bias="none",
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
    use_gradient_checkpointing="unsloth",  # Reduces memory usage
    target_modules = "all-linear", # Optional now! Can specify a list if needed
    task_type="SEQ_CLS",
)

print("model parameters:" + str(sum(p.numel() for p in model.parameters())))
# make all parameters trainable
# for param in model.parameters():
#     param.requires_grad = True

==((====))==  Unsloth 2025.10.12: Fast Modernbert patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33+5d4b92a5.d20251029. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Unsloth: Making `model.base_model.model.model` require gradients
Unsloth: Upcasting `base_model.model.classifier` from float16 to float32 since it's in `modules_to_save`. Also allowing gradients.
model parameters:403032068


In [7]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples):
    # return tokenizer(examples['text'])
    return tokenizer(examples['text'], max_length=512, truncation=True, padding=False)  


train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

print(test_dataset)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 25000
})


In [8]:
training_args = TrainingArguments(
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,

    warmup_steps=100,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_torch",
    learning_rate=2e-5,
    weight_decay=0.001,
    lr_scheduler_type="linear",
    seed=3407,
    num_train_epochs=3,  # bert-style models usually need more than 1 epoch
    save_strategy="epoch",
    # report_to="wandb",
    report_to="none",
    # group_by_length=True,
    # eval_strategy="no",
    eval_strategy="epoch",
    # eval_steps=0.25,
    logging_strategy="steps",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer_stats = trainer.train()

print(trainer_stats)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3 | Total steps = 939
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 1 x 1) = 64
 "-____-"     Trainable parameters = 7,198,722 of 403,032,068 (1.79% trained)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.169800,0.165361,0.946400
2,0.122200,0.147218,0.949200
3,0.117800,0.140501,0.952400


Unsloth: Will smartly offload gradients to save VRAM!
TrainOutput(global_step=939, training_loss=0.22281532997267442, metrics={'train_runtime': 9160.06, 'train_samples_per_second': 6.55, 'train_steps_per_second': 0.103, 'total_flos': 6.478022909952e+16, 'train_loss': 0.22281532997267442, 'epoch': 3.0})


In [9]:
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

model = model.eval()
FastLanguageModel.for_inference(model)

# 自定义collate函数处理变长序列
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    
    # 找到最大长度进行填充
    max_len = max(len(seq) for seq in input_ids)
    
    # 填充序列
    padded_input_ids = []
    padded_attention_mask = []
    
    for seq, mask in zip(input_ids, attention_mask):
        pad_len = max_len - len(seq)
        padded_input_ids.append(seq + [0] * pad_len)  # 用0填充
        padded_attention_mask.append(mask + [0] * pad_len)  # 用0填充
    
    return {
        'input_ids': torch.tensor(padded_input_ids),
        'attention_mask': torch.tensor(padded_attention_mask)
    }

# 创建DataLoader
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    shuffle=False
)

all_predictions = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        inputs = {
            'input_ids': batch['input_ids'].to(model.device),
            'attention_mask': batch['attention_mask'].to(model.device)
        }
        
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
        all_predictions.extend(predictions)

test_pred = np.array(all_predictions).flatten()

result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/kaggle/working/modernbert_unsloth.csv", index=False, quoting=3)

  0%|          | 0/782 [00:00<?, ?it/s]